In [2]:
import findspark  #Findspark can add a startup file to the current IPython profile
findspark.init()  #to make pyspark importable as a regular library.
import pyspark

In [3]:
from pyspark.sql import SparkSession # A SparkSession can be used create DataFrame, execute SQL over tables, cache tables, and read parquet files
from pyspark.sql.functions import col, count, desc, sum


In [13]:
if __name__ == '__main__':
    spark = SparkSession.builder.getOrCreate()  #getOrCreate () Gets an existing SparkSession or,if there is no existing one, creates a new one based on the options set in this builder
    

In [15]:
# Reading Parquet file

df_1 = spark.read.parquet("D:/Data/confidential.snappy.parquet") #
df_1.show(5)
    

+--------+--------------+------------+------------+------------+-----+------------+-------+----------------------------+--------------+---------+--------+-----------+------------+-----------+-------------+-----------------+-----------------+----------------+
|      ID|Isconfidential| ProjectName|      Street|        City|State|     Zipcode|Country|LEEDSystemVersionDisplayName|PointsAchieved|CertLevel|CertDate|IsCertified|  OwnerTypes|GrossSqFoot|TotalPropArea|     ProjectTypes|OwnerOrganization|RegistrationDate|
+--------+--------------+------------+------------+------------+-----+------------+-------+----------------------------+--------------+---------+--------+-----------+------------+-----------+-------------+-----------------+-----------------+----------------+
|10000001|           Yes|Confidential|Confidential|Confidential|   IN|Confidential|     US|                 LEED-NC 2.0|          null|     null|    null|         No|Confidential|     291000|       130637|Commercial Office|

In [18]:
# provides an easily readable view of the DataFrame schema 
df_1.printSchema()  

root
 |-- ID: string (nullable = true)
 |-- Isconfidential: string (nullable = true)
 |-- ProjectName: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- LEEDSystemVersionDisplayName: string (nullable = true)
 |-- PointsAchieved: string (nullable = true)
 |-- CertLevel: string (nullable = true)
 |-- CertDate: string (nullable = true)
 |-- IsCertified: string (nullable = true)
 |-- OwnerTypes: string (nullable = true)
 |-- GrossSqFoot: string (nullable = true)
 |-- TotalPropArea: string (nullable = true)
 |-- ProjectTypes: string (nullable = true)
 |-- OwnerOrganization: string (nullable = true)
 |-- RegistrationDate: string (nullable = true)



In [20]:
# Reading CSV file
df_2 = spark.read.csv("D:/Data/nonConfidential.csv", header=True)
df_2.show(5)

+--------+--------------+--------------------+--------------------+----------+-----+-------+-------+----------------------------+--------------+---------+---------------+-----------+----------------+-----------+-------------+--------------------+--------------------+----------------+
|      ID|Isconfidential|         ProjectName|              Street|      City|State|Zipcode|Country|LEEDSystemVersionDisplayName|PointsAchieved|CertLevel|       CertDate|IsCertified|      OwnerTypes|GrossSqFoot|TotalPropArea|        ProjectTypes|   OwnerOrganization|RegistrationDate|
+--------+--------------+--------------------+--------------------+----------+-----+-------+-------+----------------------------+--------------+---------+---------------+-----------+----------------+-----------+-------------+--------------------+--------------------+----------------+
|10000000|            No|PNC Firstside Center|21 South Putt Cor...|Pittsburgh|   PA|  15219|     US|                 LEED-NC 2.0|            33| 

In [19]:
# Provides an easily readable view of the DataFrame schema 
df_2.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Isconfidential: string (nullable = true)
 |-- ProjectName: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- LEEDSystemVersionDisplayName: string (nullable = true)
 |-- PointsAchieved: string (nullable = true)
 |-- CertLevel: string (nullable = true)
 |-- CertDate: string (nullable = true)
 |-- IsCertified: string (nullable = true)
 |-- OwnerTypes: string (nullable = true)
 |-- GrossSqFoot: string (nullable = true)
 |-- TotalPropArea: string (nullable = true)
 |-- ProjectTypes: string (nullable = true)
 |-- OwnerOrganization: string (nullable = true)
 |-- RegistrationDate: string (nullable = true)



In [21]:
# Combine Two DataFrame
df = df_1.unionAll(df_2)
df.show(5)

# UnionAll() function row binds two dataframe in pyspark and does not removes the duplicates this is called union all in pyspark.

+--------+--------------+------------+------------+------------+-----+------------+-------+----------------------------+--------------+---------+--------+-----------+------------+-----------+-------------+-----------------+-----------------+----------------+
|      ID|Isconfidential| ProjectName|      Street|        City|State|     Zipcode|Country|LEEDSystemVersionDisplayName|PointsAchieved|CertLevel|CertDate|IsCertified|  OwnerTypes|GrossSqFoot|TotalPropArea|     ProjectTypes|OwnerOrganization|RegistrationDate|
+--------+--------------+------------+------------+------------+-----+------------+-------+----------------------------+--------------+---------+--------+-----------+------------+-----------+-------------+-----------------+-----------------+----------------+
|10000001|           Yes|Confidential|Confidential|Confidential|   IN|Confidential|     US|                 LEED-NC 2.0|          null|     null|    null|         No|Confidential|     291000|       130637|Commercial Office|

In [22]:
 # 1. How many LEED projects are there in Virginia (including all types of project types  and versions of LEED)?
    
virginia_df = df.where(col("City").contains("Virginia"))
print(virginia_df.count())

57


In [23]:
# 2. What is the number of LEED projects in Virginia by owner type?

virginia_df.groupby("OwnerTypes").agg(count("OwnerTypes").alias("count")).show()


+--------------------+-----+
|          OwnerTypes|count|
+--------------------+-----+
|Government Use: F...|   11|
|  Federal Government|    4|
|         Profit Org.|    4|
|Educational: K-12...|    2|
|                null|    0|
|          Individual|    2|
|Corporate: Privat...|    5|
|Government Use: L...|    1|
|    State Government|    3|
|               Other|    1|
|Government Use: L...|    6|
|Educational: Coll...|    1|
|Government Use: O...|    1|
|Investor: REIT, P...|    2|
|    Local Government|    7|
|     Non-Profit Org.|    2|
+--------------------+-----+



In [11]:
#What is the total Gross Square Feet of building space that is LEED-certified in  Virginia?   

virginia_df.where(col("IsCertified") == "Yes").agg(sum("GrossSqFoot").alias("Total_GrossSqFoot")).show()



+-----------------+
|Total_GrossSqFoot|
+-----------------+
|      2.8501411E7|
+-----------------+



In [12]:
#What Zip Code in Virginia has the highest number of projects? 
zipcode_df = virginia_df.groupby("Zipcode").agg(count("Zipcode").alias("count"))
zipcode_df.orderBy(col("count").desc()).show(1)

+-------+-----+
|Zipcode|count|
+-------+-----+
|  23462|   11|
+-------+-----+
only showing top 1 row

